In [20]:
%matplotlib inline

In [21]:
# import tensorflow as tf
from math import *
import numpy as np
from random import shuffle
from pylab import *
from multiprocessing import Pool

In [38]:

def device_info_load():
    device_models = {}
    
    for fn in ['contest_dataset_app_device/device000000_0', 'contest_dataset_app_device/device000001_0']: 
        with open(fn) as file_device:
            models = {}
            while True:
                x = file_device.readline()
                if len(x) == 0:
                    break
                user_id, device_model, device_grand = x.split()
                device_models[user_id] = device_model
                
    return device_models

device_models = device_info_load()
print len(device_models)

12439408


In [35]:
from random import shuffle

def read_user_labels():
    user_labels = {}

    for fn in ['contest_dataset_label/label000000_0', 'contest_dataset_label/label000001_0']:
        with open(fn) as file_user_label:
            lines = file_user_label.readlines()
            shuffle(lines)
            for i, x in enumerate(lines):
                user_id, gender, user_age = x.split()
                user_labels[user_id] = [gender, user_age]

    return user_labels

user_labels = read_user_labels()
print len(user_labels)

11439408


In [40]:
#freq_gender = np.zeros((250, 250), np.float32)
#freq_age = np.zeros((250, 250), np.float32)
freq_gender = np.zeros((250, 2), np.float32)
freq_age = np.zeros((250, 7), np.float32)
for user_id in user_labels.keys():
    if user_id not in device_models:
        continue
    gender, age = user_labels[user_id]
    gender = int(gender) - 1
    age = int(age) - 1
    device_model = int(device_models[user_id])
    freq_gender[device_model][gender] += 1
    freq_age[device_model][age] += 1

In [41]:
freq_gender_eps = freq_gender + 1e-6
freq_age_eps = freq_age + 1e-6

distribution_gender = freq_gender_eps / (np.sum(freq_gender_eps, axis=1, keepdims=True))
distribution_age = freq_age_eps / (np.sum(freq_age_eps, axis=1, keepdims=True))

In [42]:
distribution_age

count = 0
for gender, _ in user_labels.values():
    if gender == '2':
        count += 1
print count


3804980


In [43]:
testset = []
with open('contest_testset/testset000000_0') as file_test:
    for x in file_test.readlines():
        testset += [x[:-1]]


In [48]:
fallback_gender = np.array([0.3817514, 0.1901782])
fallback_age = np.array([0.181793, 0.15182, 0.128824, 0.234479, 0.193585, 0.06086, 0.04864])
fallback_gender /= np.sum(fallback_gender)
fallback_age /= np.sum(fallback_age)

def get_prediction(user_id):
    if user_id in device_models:
        device_model = device_models[user_id]
        cur_gender = distribution_gender[int(device_model)]
        cur_age = distribution_age[int(device_model)]
    else:
        cur_gender = fallback_gender
        cur_age = fallback_age
    return cur_gender, cur_age

with open('output2.csv', 'w') as fout:
    print >> fout, 'user_id,gender_1,gender_2,age_g1,age_g2,age_g3,age_g4,age_5,age_g6,age_7'

    for test_id in testset:
        cur_gender, cur_age = get_prediction(test_id)

        print >> fout, '%s,%.6f,%.6f,%.6f,%.6f,%.6f,%.6f,%.6f,%.6f,%.6f' % (test_id, cur_gender[0], cur_gender[1], cur_age[0], cur_age[1], cur_age[2], cur_age[3], cur_age[4], cur_age[5], cur_age[6])

In [ ]:
def load_install():
    D = {}
    for i in range(lo, hi):
        fn = 'contest_dataset_app_install/part-r-%05d' % (i)
        print fn
        with open(fn) as fin:
            for x in fin:
                user_id, app_id = x.split()
                D[app_id] += 1
    return D